In [ ]:
import pandas as pd
import numpy as np
import sys

sys.path.append("/Users/nwong/Workspace/Projects/tdt4173_project/src")

from feature_engineering.bus_stop_features import *
from feature_engineering.demographic_features import *
from feature_engineering.demographic_features import *
from feature_engineering.store_features import *

In [ ]:
# Import datasets
bus_stops_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/busstops_norway.csv")
grunnkrets_age_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_age_distribution.csv")
grunnkrets_household_pop_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_households_num_persons.csv")
grunnkrets_household_inc_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_income_households.csv")
grunnkrets_norway_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/grunnkrets_norway_stripped.csv")
plaace_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/plaace_hierarchy.csv")
stores_extra_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_extra.csv")
stores_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_train.csv")
stores_test_df = pd.read_csv("/Users/nwong/Workspace/Projects/tdt4173_project/data/raw/stores_test.csv")

stores_df = stores_df[stores_df['year'] == 2016].merge(plaace_df, on="plaace_hierarchy_id", how="left")
stores_extra_df = stores_extra_df[stores_extra_df['year'] == 2016]

# grunnkrets_norway_df = grunnkrets_norway_df[grunnkrets_norway_df['year'] == 2016]
# bus_stops_df = bus_stops_lat_lon(bus_stops_df)

# merged_df = stores_df.merge(grunnkrets_norway_df, how="left", on="grunnkrets_id").merge(plaace_df, how="left", on="plaace_hierarchy_id")

# geo_groups = ['grunnkrets_name','district_name','municipality_name']

# store_types = ['lv1_desc', 'lv2_desc', 'lv3_desc', 'lv4_desc']

# stop_importance_levels = list(bus_stops_df['importance_level'].unique())

In [ ]:
stores_df = stores_df.merge(plaace_df, on="plaace_hierarchy_id", how="left")
stores_in_radius(stores_df, stores_df, radius=0.1, store_type_group='lv1_desc')

In [ ]:
# pop_per_store_type = population_per_store_type_grouped_by_geo_groups(stores_test_df, plaace_df, grunnkrets_norway_df, grunnkrets_age_df, geo_groups=geo_groups, store_types=store_types, agg_string="pop_per_num_stores")
# pop_density = population_density_grouped_by_geo_group(stores_test_df, grunnkrets_age_df, grunnkrets_norway_df)
# pop_count_df = population_count_grouped_by_geo_group(stores_test_df, grunnkrets_age_df, grunnkrets_norway_df, geo_groups=geo_groups)
# age_dist_df = age_dist_by_geo_group(stores_test_df, grunnkrets_age_df, grunnkrets_norway_df)
# house_hold_dist = household_dist_by_geo_group(stores_test_df, grunnkrets_household_pop_df, grunnkrets_norway_df)

# is_mall_df = is_mall_only(stores_test_df)
# is_chain_df = is_chain_only(stores_test_df)
# store_types_count = store_types_all_count_by_geo_groups(stores_test_df, plaace_df, grunnkrets_norway_df, store_types=store_types, geo_groups=geo_groups)
# store_types_revenue = store_types_all_revenue_by_geo_groups(stores_df, stores_test_df, plaace_df, grunnkrets_norway_df, store_types=store_types, geo_groups=geo_groups)
# store_radius = stores_in_radius_by_type(stores_test_df, plaace_df, store_types=store_types, radius = 0.1)

# busstop_distance = bus_stops_distance_by_importance(stores_test_df, bus_stops_df, stop_importance_levels = stop_importance_levels)
# busstop_radius = bus_stops_in_radius_by_importance(stores_test_df, bus_stops_df, stop_importance_levels=stop_importance_levels, radius = 0.1)

# df = (pop_count_df.merge(age_dist_df, how = "left", on = "store_id")
#     .merge(house_hold_dist, how = "left", on ="store_id")
#     .merge(pop_per_store_type, how = "left", on = "store_id")
#     .merge(pop_density, how = "left", on  = "store_id")
#     .merge(is_mall_df, how = "left", on = "store_id")
#     .merge(is_chain_df, how = "left", on = "store_id")
#     #.merge(income_df, how = "left", on = "store_id")
#     .merge(store_types_count, how ="left", on = "store_id")
#     .merge(store_types_revenue, how = "left", on = "store_id")
#     #.merge(id_and_revenue_df, how = "left", on = "store_id")
#     .merge(store_radius, how = "left", on = "store_id")
#     .merge(busstop_distance,how = "left", on = "store_id")
#     .merge(busstop_radius, how ="left", on ="store_id" )
# )

# df

In [ ]:
stores = stores_df.drop("revenue", axis=1)
stores_labels = stores_df['revenue'].copy()

In [ ]:
stores

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
stores_num = stores.select_dtypes(include=[np.number])
X = imputer.fit_transform(stores_num)
stores_tr = pd.DataFrame(X, columns=stores_num.columns, index=stores_num.index)
stores_tr

In [ ]:
from sklearn.preprocessing import OneHotEncoder
stores_cat = stores[['chain_name']]

cat_encoder = OneHotEncoder()
stores_cat_lhot = cat_encoder.fit_transform(stores_cat)
pd.DataFrame(stores_cat_lhot.toarray(), columns=cat_encoder.get_feature_names_out(), index=stores_cat.index)

In [ ]:
# Transform feature with a heavy tail prior to scaling. Otherwise, the values will get squashed by outliers
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

cat_attribs = ["sales_channel_name", "chain_name", "mall_name"]

cat_pipeline = make_pipeline(
    # SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse=False)
)

preprocessing = ColumnTransformer([
    ("cat", cat_pipeline, cat_attribs),
])

stores_preped = preprocessing.fit_transform(stores[cat_attribs])
pd.DataFrame(stores_preped, columns=preprocessing.get_feature_names_out(), index=stores.index)

In [ ]:

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
stores_num_std_scaled = std_scaler.fit_transform(stores_num)
pd.DataFrame(stores_num_std_scaled, columns=stores_num.columns, index=stores_num.index)

In [ ]:
from sklearn.preprocessing import FunctionTransformer
ratio_transformer = FunctionTransformer(lambda X: X[:, [0]] / X[:, [1]])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state
        
    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self
    
    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f'Cluster {i} similarity' for i in range(self.n_clusters)]

In [ ]:
from matplotlib import pyplot as plt
cluster_simil = ClusterSimilarity(n_clusters=15, gamma=1.,random_state=42)
cluster_simil.fit_transform(stores[['lat', 'lon']], sample_weight=stores_labels)
print(pd.DataFrame(cluster_simil.kmeans_.cluster_centers_))
plt.scatter(pd.DataFrame(cluster_simil.kmeans_.cluster_centers_)[1], pd.DataFrame(cluster_simil.kmeans_.cluster_centers_)[0])

In [ ]:
stores = store_types_all_count_by_geo_groups(stores_df, plaace_df, grunnkrets_norway_df, store_types, geo_groups)
population = population_grouped_by_geo_group(stores_df, grunnkrets_age_df, grunnkrets_norway_df)

In [ ]:
stores

In [ ]:
population

In [ ]:
def store_density_all_by_geo_groups(stores_df, population_df):
    merged_df = pd.concat([stores_df, population_df], axis=1)
    print(merged_df)
    merged_df[['a', 'b', 'c', 'd']] = merged_df[['grunnkrets_name_lv1_desc','grunnkrets_name_lv2_desc', 'grunnkrets_name_lv3_desc', 'grunnkrets_name_lv4_desc']] / merged_df['grunnkrets_name_population_count'].to_numpy()
    return merged_df

In [ ]:
store_density_all_by_geo_groups(stores, population)

In [ ]:
def population_per_store_types(stores_df, plaace_hierarchy, grunnkrets_df, age_df, agg_name, geo_group, store_type_group): 
    grunnkrets_df_2016 = grunnkrets_df[grunnkrets_df["year"] == 2016]
    num_stores_types_by_geo_group = store_types_count_by_geo_group(stores_df, plaace_hierarchy, grunnkrets_df_2016, agg_name, geo_group, store_type_group)
    pop_grouped_by_geo = population_grouped(age_df, grunnkrets_df_2016, geo_group)
    combined_df = num_stores_types_by_geo_group.merge(pop_grouped_by_geo, how = "left", on = geo_group)
    combined_df["population_per_num_store"] = combined_df["population_count"] / combined_df[agg_name]

In [ ]:
population_per_store_types(stores_df, plaace_hierarchy)

In [ ]:
bus_stops_distance_by_importance(stores_df, bus_stops_df, stop_importance_levels)

In [ ]:
bus_stops_in_radius_by_importance(stores_df, bus_stops_df, stop_importance_levels, radius=0.05)

In [ ]:
stores_in_radius_by_type(stores_df, plaace_df, store_types, radius=0.05)

In [ ]:
store_types_all_count_by_geo_groups(stores_df, plaace_df, grunnkrets_norway_df, store_types, geo_groups)

In [ ]:
store_types_all_revenue_by_geo_groups(stores_df, plaace_df, grunnkrets_norway_df, store_types, geo_groups)